To DO

+ ~~change landing page~~
+ ~~DONE add loading options to simulation; must explore useAsyncQueue NOPE (AsyncQueue too different from AsyncState)~~
+ ~~DONE add table; use icons for table segments; need to have column groups~~
+ ~~DONE!   need to add charts~~

+ ~~localstorage for chart values~~
+ ~~add env statement~~
+ ~~improve Chart aesthetics~~
    ~~+ window or tiles or windows with tiles~~
    ~~+ stack legend~~
    ~~+ use theme-related colors~~
    ~~+ pass appropriate x- and y-label values~~
+ ~~standardize container sizes and alignments~~
+ ~~improve Table aesthetics~~
    + ~~add units for rows in Summary~~
+ ~~use Celery and Redis for async tasks to save monthly / quarterly / annual versions of statements~~
+ ~~added celery autoreload~~
+ ~~MAKE NOTE THAT CHARTS ARE MONTHLY!!!!~~
+ ~~check all math~~
    + ~~why is block schedule wrong?~~
        + ~~had placeholder for current block~
    + ~~number of miners???~~
    + ~~cost of miners~~
    + ~~Why do roi and istat not overlap in terms of number of periods??????~~
    + ~~no pool fees?~~
+ ~~check if statement object exist, if so, get monthly and summary only; should be much faster~~
+ ~~add current BTC environment modal~~
    + ~~feed-in live environment to enviroment assumptions~~
+ ~~no scaling done for Heat rejection products????~~
+ ~~improve aesthetics of environment section~~
    + ~~improve chart layout~~
    + ~~improve summary layout~~
+ ~~enviro aesthetics~~
    + ~~transaction fees y-axis range needs fixing~~
    + ~~enviro select load ... no user feedback~~
+ current
    + ~~improve chart dropdown~~
    + ~~confirm chart data accurate~~
    + ~~add feature to change period for chart~~
    + ~~add support for tuscaloosa temps~~
    + ~~fix infra inputs section~~
    + ~~clear env should unlock all~~
    + ~~Suspense to load summary correclty~~
    + ~~load params into factorForm properly ~~
    + ~~create multiple statements; overwrite existing Sim~~
    + ~~no charts showing; multiple projects not working~~
    + ~~progress bar for periodic statements~~
    + ~~need legend on charts~~
    + ~~don't overwrite not working~~
    + ~~first time after auto-scale values don't change agin~~
    + ~~delete project not working~~
    + ~~show efficiency of mining rigs~~
    + ~~add project f/s~~
    + ~~why no concurrency on celery tasks?~~
+ major Simulation Improvements
    + ~~need presentation package!!!!~~
    + add opex projection module
    + option to scale from components up to power capacity
    + compare across environments
    + add temperature impacts
        + currently using Student's T to simulate temps but need to clip at .25% and 99.5% to eliminate extreme values;
        + see products/serializers line 259/260 in DryBulbSimulationSerializer
        + is there a better distribution? faster way to do all stations?
        + need to use time-series ....
    + need to add curtailment / switching on-off of miners
    + need to feed in variable chip temp
    + need to account for delta T at higher overclocking
    + additional metrics:
        + compare to BTC HODL strategy
        + percentage revenue from appreciation vs. fees
        + hash share should be percentage of actual hash rate, not difficulty
+ aesthetic / input imporvements
    + fix Epoch alignmnet for block schedule
    + clean-up axios error handling
    + add new mining rig not working; buffer input not working
    + ~~allow manual scaling and pricing of products~~
    + ~~need full statements and charts for each project~~
    + ~~remove all charts not being used~~
    + ~~flag form changes for save~~
    + ~~input title at top~~
    + ~~improve infra section and remove errors~~
    + ~~ProjectSim CRUD at point of sale ... need to clean up combobox interaction~~
+ additional Module improvements
    + price updates for miners
    + add Bitcoin mining demo
    + improve infra product database

+ drillbit improvements
    + time unit not working as expected

+ code efficiency 
    + ~~push M/Q/A statement creation INTO the serializer so that finstat resample can be used; how to get task_ids ?????~~
    + need better atomicity of database transactions

+ try to upgrade python and javascript package versions
+ push live to google
+ testing for drillbit
+ testing for drillbit-vj
+ testing for drillbit-dj
+ updates to finstat

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'drillbit_dj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
import numpy as np
import pandas as pd

In [4]:
from drillbit import BitcoinEnvironmentUtility
from drillbit.__new_objects import Rig as RigManager, Product as ProductManager, \
    Cooling as CoolingManager, Project as ProjectManager, \
    OverClock, RigOperator
from drillbit.__new_units__ import AbstractBaseUnit, MagnitudeTable, Hashes, HashRate, \
    Time, Power, Energy, Efficiency, EnergyPrice


from environment.serializers import EnvironmentSerializer as EnviroSer, BlockScheduleSerializer as BSchedSer, \
    BitcoinPriceSerializer as BPSer, TransactionFeesSerializer as FeeSer, HashRateSerializer as HRSer
from environment.models import Environment, BlockSchedule, BitcoinPrice, TransactionFees, HashRate as HashRateModel

from products.models import Rig, Cooling, HeatRejection, Electrical, RejectionCurve
from products.serializers import RigSerializer, CoolingSerializer, HeatRejectionSerializer, \
    ElectricalSerializer, RejectionCurveForProductSerializer as CurveSerializer

from projects.models import RigForProject, InfraForProject, Project, Projects, \
    ProjectStatement, ProjectSimulation, ProjectStatementSummary
from projects.serializers import RigForProjectSerializer, InfraForProjectSerializer, ProjectSerializer, ProjectsSerializer, \
    ProjectScalingSerializer, ProjectCostsSerializer, ProjectStatementSerializer, \
    ProjectStatementSummarySerializer, fit_temperature_to_environment, \
    load_json_block_statement_and_resample

from projects.tasks import create_statements_for_given_project, create_statement_for_given_frequency

In [5]:
import numpy as np
from tqdm.auto import tqdm, trange
import pandas as pd
import numpy_financial as npf

import finstat as fs

from drillbit.statements.funcs import total_energy, win_percentage, \
    expected_difficulty, hashes_to_hash_rate, win_percentage
from drillbit.statements.statements import ProjectTemplate

from products.models import WeatherStation, WeatherData, save_dry_bulb_frequency, save_diurnal
from products.serializers import expected_loss_from_insufficient_cooling_capacity, \
    RejectionTemperatureImpactSerializer as RejTempSer, \
    RejectionTemperaturePaybackSerializer as RejPayBackSer, \
    DryBulbSimulationSerializer as DBSimSer

In [6]:
from drillbit.statements.statements import init_environment, ProjectTemplate, ROITemplate, \
    analysis

In [ ]:
# now i want to add an "environments" group